In [1]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten

#加载训练数据
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
x_train = []
y_train = []
for i in range(1, 6):
    batch = unpickle(f'cifar-10-batches-py/data_batch_{i}')
    x_train.append(batch[b'data'])
    y_train.append(batch[b'labels'])
x_train = np.concatenate(x_train)
y_train = np.concatenate(y_train)
#加载测试数据
test_batch = unpickle('cifar-10-batches-py/test_batch')
x_test = test_batch[b'data']
y_test = test_batch[b'labels']
#交换通道
x_train = x_train.reshape(x_train.shape[0], 3, 32, 32).transpose(0, 2, 3, 1)
x_test = x_test.reshape(x_test.shape[0], 3, 32, 32).transpose(0, 2, 3, 1)
#调整输入尺寸以适应ResNet50
x_train = tf.image.resize(x_train, (48, 48)).numpy()
x_test = tf.image.resize(x_test, (48, 48)).numpy()
#归一化数据
x_train, x_test = x_train / 255.0, x_test / 255.0
#将标签列表转换为NumPy数组
y_train = np.array(y_train)
y_test = np.array(y_test)
#构建ResNet模型
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(48, 48, 3))
#冻结ResNet模型的权重
for layer in base_model.layers:
    layer.trainable = False
#添加自定义层
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)
#构建最终模型
model = Model(inputs=base_model.input, outputs=predictions)
#编译模型
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
#训练模型
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))
#评估模型
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc}')

Epoch 1/10
1563/1563 [==============================] - 171s 108ms/step - loss: 1.9323 - accuracy: 0.2863 - val_loss: 1.8157 - val_accuracy: 0.3340
Epoch 2/10
1563/1563 [==============================] - 158s 101ms/step - loss: 1.7498 - accuracy: 0.3642 - val_loss: 1.6841 - val_accuracy: 0.3984
Epoch 3/10
1563/1563 [==============================] - 172s 110ms/step - loss: 1.6908 - accuracy: 0.3893 - val_loss: 1.7298 - val_accuracy: 0.3662
Epoch 4/10
1563/1563 [==============================] - 166s 106ms/step - loss: 1.6638 - accuracy: 0.4039 - val_loss: 1.6114 - val_accuracy: 0.4184
Epoch 5/10
1563/1563 [==============================] - 161s 103ms/step - loss: 1.6426 - accuracy: 0.4115 - val_loss: 1.5963 - val_accuracy: 0.4311
Epoch 6/10
1563/1563 [==============================] - 168s 108ms/step - loss: 1.6192 - accuracy: 0.4178 - val_loss: 1.6285 - val_accuracy: 0.4142
Epoch 7/10
1563/1563 [==============================] - 162s 103ms/step - loss: 1.6056 - accuracy: 0.4240 - val_